In [ ]:
!pip -q install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import hashlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, average_precision_score

from scipy import stats

# TabNet
from pytorch_tabnet.tab_model import TabNetClassifier

In [ ]:
SEEDS = [42, 52, 62, 72, 82]
TEST_SIZE = 0.30
VAL_SIZE  = 0.20

TABNET_PARAMS = dict(
    max_epochs=10,
    patience=10,
    batch_size=512,
    virtual_batch_size=128,
    num_workers=0
)

BASE_ORIGINAL = "/content/OroginalDatasetBalanced.csv"

In [ ]:
def apply_hashing(df):
    for col in ['Source IP', 'Destination IP', 'Flow ID']:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
    return df

def get_numeric_cols(df, label_col="Label"):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    numeric_cols = numeric_cols.drop(label_col)
    return numeric_cols

def apply_noise_only(df, numeric_cols, factor=0.35):
    for col in numeric_cols:
        std = df[col].std()
        noise = np.random.normal(0, factor * std, size=len(df))
        df[col] = (df[col] + noise).clip(lower=0)
    return df

def apply_binning_only(df, numeric_cols, q=5):
    for col in numeric_cols:
        try:
            df[col] = pd.qcut(df[col], q=q, labels=False, duplicates='drop')
        except:
            pass
    return df

def apply_masking_only(df, numeric_cols, mask_prob=0.15):
    for col in numeric_cols:
        mask = np.random.rand(len(df)) < mask_prob
        df.loc[mask, col] = 0
    return df

def apply_permutation_only(df, numeric_cols):
    for col in numeric_cols:
        df[col] = np.random.permutation(df[col].values)
    return df

In [ ]:
def build_ablation_csvs(base_csv):
    df0 = pd.read_csv(base_csv)

    # Encode labels (exactly your approach)
    df0["Label"] = df0["Label"].map({"Benign": 0, "Malicious": 1})
    df0.dropna(subset=["Label"], inplace=True)

    # Hashing (consistent with your obfuscation scripts)
    df0 = apply_hashing(df0)

    numeric_cols = get_numeric_cols(df0, "Label")

    # Use ONE fixed seed for obfuscation generation
    np.random.seed(0)

    # Noise-only
    df_noise = apply_noise_only(df0.copy(), numeric_cols, factor=0.3)
    df_noise.to_csv("/content/Ablation_NoiseOnly.csv", index=False)

    # Binning-only
    df_bin = apply_binning_only(df0.copy(), numeric_cols, q=5)
    df_bin.to_csv("/content/Ablation_BinningOnly.csv", index=False)

    # Masking-only
    df_mask = apply_masking_only(df0.copy(), numeric_cols, mask_prob=0.2)
    df_mask.to_csv("/content/Ablation_MaskingOnly.csv", index=False)

    # Permutation-only
    df_perm = apply_permutation_only(df0.copy(), numeric_cols)
    df_perm.to_csv("/content/Ablation_PermutationOnly.csv", index=False)

    print("✅ Saved ablation datasets:",
          "Ablation_NoiseOnly.csv, Ablation_BinningOnly.csv, Ablation_MaskingOnly.csv, Ablation_PermutationOnly.csv")

build_ablation_csvs(BASE_ORIGINAL)

✅ Saved ablation datasets: Ablation_NoiseOnly.csv, Ablation_BinningOnly.csv, Ablation_MaskingOnly.csv, Ablation_PermutationOnly.csv


In [ ]:
def load_and_preprocess(csv_path, label_col="Label"):
    df = pd.read_csv(csv_path)
    if label_col not in df.columns:
        raise ValueError(f"Label column '{label_col}' not found in {csv_path}")

    # ---- Label handling (supports [0,1] or ['Benign','Malicious']) ----
    if df[label_col].dtype == object:
        df = df[df[label_col].isin(["Benign", "Malicious"])].copy()
        y = df[label_col].map({"Benign": 0, "Malicious": 1}).astype(int).to_numpy()
    else:
        df = df[df[label_col].isin([0, 1])].copy()
        y = df[label_col].astype(int).to_numpy()

    # ---- Numeric features only ----
    X = df.drop(columns=[label_col], errors="ignore").select_dtypes(include=[np.number]).copy()

    # Drop constant columns
    if len(X.columns) == 0:
        raise ValueError("No numeric features found after selecting numeric columns.")
    X = X.loc[:, (X != X.iloc[0]).any()]

    # Clean inf/nan
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    X.fillna(0, inplace=True)

    return X.to_numpy(), y, X.columns.tolist()

In [ ]:
def train_eval_tabnet(X, y, seed=42):
    # 1) Train/Test split
    X_train_full, X_test, y_train_full, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, stratify=y, random_state=seed
    )

    # 2) Train/Val split (from train only)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full, test_size=VAL_SIZE, stratify=y_train_full, random_state=seed
    )

    # 3) Scale: fit on TRAIN only
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val   = scaler.transform(X_val)
    X_test  = scaler.transform(X_test)

    # 4) Train TabNet using VAL as eval_set (no test leakage)
    clf = TabNetClassifier(seed=seed)
    clf.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_name=["val"],
        eval_metric=["accuracy"],
        **TABNET_PARAMS
    )

    # 5) Final evaluation on TEST only
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred, average="macro")
    roc = roc_auc_score(y_test, y_prob)
    pr  = average_precision_score(y_test, y_prob)
    cm  = confusion_matrix(y_test, y_pred)

    return acc, f1, roc, pr, cm

In [ ]:
DATASETS = {
    "Original": BASE_ORIGINAL,
    "NoiseOnly": "/content/Ablation_NoiseOnly.csv",
    "BinningOnly": "/content/Ablation_BinningOnly.csv",
    "MaskingOnly": "/content/Ablation_MaskingOnly.csv",
    "PermutationOnly": "/content/Ablation_PermutationOnly.csv",
}

In [ ]:
results = []

for name, path in DATASETS.items():
    print("\n" + "="*80)
    print(f"DATASET: {name}")
    print(f"FILE: {path}")
    print("="*80)

    X, y, feats = load_and_preprocess(path)

    for seed in SEEDS:
        acc, f1, roc, pr, cm = train_eval_tabnet(X, y, seed=seed)
        results.append([name, seed, acc, f1, roc, pr])

summary = pd.DataFrame(
    results,
    columns=["Dataset", "Seed", "Accuracy", "F1_macro", "ROC_AUC", "PR_AUC"]
)
summary


DATASET: Original
FILE: /content/OroginalDatasetBalanced.csv


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.56492 | val_accuracy: 0.82643 |  0:00:00s
epoch 1  | loss: 0.27631 | val_accuracy: 0.90286 |  0:00:01s
epoch 2  | loss: 0.20277 | val_accuracy: 0.94071 |  0:00:01s
epoch 3  | loss: 0.15406 | val_accuracy: 0.94071 |  0:00:02s
epoch 4  | loss: 0.1162  | val_accuracy: 0.95214 |  0:00:02s
epoch 5  | loss: 0.10919 | val_accuracy: 0.97143 |  0:00:02s
epoch 6  | loss: 0.11084 | val_accuracy: 0.97786 |  0:00:03s
epoch 7  | loss: 0.09537 | val_accuracy: 0.97571 |  0:00:03s
epoch 8  | loss: 0.08028 | val_accuracy: 0.97143 |  0:00:04s
epoch 9  | loss: 0.07261 | val_accuracy: 0.95929 |  0:00:04s
Stop training because you reached max_epochs = 10 with best_epoch = 6 and best_val_accuracy = 0.97786


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.5895  | val_accuracy: 0.76714 |  0:00:00s
epoch 1  | loss: 0.32703 | val_accuracy: 0.91071 |  0:00:01s
epoch 2  | loss: 0.22179 | val_accuracy: 0.94286 |  0:00:01s
epoch 3  | loss: 0.12366 | val_accuracy: 0.97143 |  0:00:02s
epoch 4  | loss: 0.11423 | val_accuracy: 0.97214 |  0:00:03s
epoch 5  | loss: 0.07975 | val_accuracy: 0.97786 |  0:00:03s
epoch 6  | loss: 0.07099 | val_accuracy: 0.95929 |  0:00:03s
epoch 7  | loss: 0.06708 | val_accuracy: 0.96143 |  0:00:04s
epoch 8  | loss: 0.07176 | val_accuracy: 0.96214 |  0:00:04s
epoch 9  | loss: 0.06086 | val_accuracy: 0.975   |  0:00:05s
Stop training because you reached max_epochs = 10 with best_epoch = 5 and best_val_accuracy = 0.97786


...

epoch 0  | loss: 0.77149 | val_accuracy: 0.51929 |  0:00:00s
epoch 1  | loss: 0.70295 | val_accuracy: 0.52429 |  0:00:01s
epoch 2  | loss: 0.69858 | val_accuracy: 0.49714 |  0:00:01s
epoch 3  | loss: 0.69593 | val_accuracy: 0.50929 |  0:00:02s
epoch 4  | loss: 0.69422 | val_accuracy: 0.48714 |  0:00:03s
epoch 5  | loss: 0.69421 | val_accuracy: 0.49071 |  0:00:03s
epoch 6  | loss: 0.69265 | val_accuracy: 0.48643 |  0:00:03s
epoch 7  | loss: 0.69315 | val_accuracy: 0.48357 |  0:00:04s
epoch 8  | loss: 0.6929  | val_accuracy: 0.49    |  0:00:04s
epoch 9  | loss: 0.69155 | val_accuracy: 0.48357 |  0:00:05s
Stop training because you reached max_epochs = 10 with best_epoch = 1 and best_val_accuracy = 0.52429


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.80065 | val_accuracy: 0.49857 |  0:00:00s
epoch 1  | loss: 0.7204  | val_accuracy: 0.48357 |  0:00:00s
epoch 2  | loss: 0.70128 | val_accuracy: 0.49357 |  0:00:01s
epoch 3  | loss: 0.69826 | val_accuracy: 0.50929 |  0:00:01s
epoch 4  | loss: 0.69446 | val_accuracy: 0.50643 |  0:00:02s
epoch 5  | loss: 0.69326 | val_accuracy: 0.505   |  0:00:02s
epoch 6  | loss: 0.69313 | val_accuracy: 0.51643 |  0:00:03s
epoch 7  | loss: 0.6929  | val_accuracy: 0.49643 |  0:00:03s
epoch 8  | loss: 0.69279 | val_accuracy: 0.48786 |  0:00:04s
epoch 9  | loss: 0.69276 | val_accuracy: 0.50786 |  0:00:04s
Stop training because you reached max_epochs = 10 with best_epoch = 6 and best_val_accuracy = 0.51643


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


,Dataset,Seed,Accuracy,F1_macro,ROC_AUC,PR_AUC
0,Original,42,0.978667,0.978667,0.995776,0.995355
1,Original,52,0.975000,0.974999,0.994232,0.993756
2,Original,62,0.974667,0.974662,0.995375,0.995980
3,Original,72,0.972667,0.972666,0.994786,0.993895
4,Original,82,0.977333,0.977333,0.994790,0.990291
5,NoiseOnly,42,0.971333,0.971333,0.993770,0.993328
6,NoiseOnly,52,0.961667,0.961666,0.989909,0.991137
7,NoiseOnly,62,0.970333,0.970330,0.993899,0.992624
8,NoiseOnly,72,0.968333,0.968332,0.995440,0.993033
9,NoiseOnly,82,0.958333,0.958301,0.989075,0.986250


In [ ]:
def mean_std_ci(x, confidence=0.95):
    x = np.array(x)
    mean = x.mean()
    std  = x.std(ddof=1)
    n    = len(x)
    h    = stats.t.ppf((1 + confidence) / 2., n-1) * std / np.sqrt(n)
    return mean, std, h

agg_rows = []

for dataset in summary["Dataset"].unique():
    df_d = summary[summary["Dataset"] == dataset]

    acc_mean, acc_std, acc_ci = mean_std_ci(df_d["Accuracy"])
    f1_mean,  f1_std,  f1_ci  = mean_std_ci(df_d["F1_macro"])
    roc_mean, roc_std, roc_ci = mean_std_ci(df_d["ROC_AUC"])
    pr_mean,  pr_std,  pr_ci  = mean_std_ci(df_d["PR_AUC"])

    agg_rows.append([
        dataset,
        acc_mean, acc_std, acc_ci,
        f1_mean,  f1_std,  f1_ci,
        roc_mean, roc_std, roc_ci,
        pr_mean,  pr_std,  pr_ci
    ])

agg = pd.DataFrame(
    agg_rows,
    columns=[
        "Dataset",
        "Accuracy_mean", "Accuracy_std", "Accuracy_95CI",
        "F1_macro_mean", "F1_macro_std", "F1_macro_95CI",
        "ROC_AUC_mean", "ROC_AUC_std", "ROC_AUC_95CI",
        "PR_AUC_mean",  "PR_AUC_std",  "PR_AUC_95CI"
    ]
)

agg

,Dataset,Accuracy_mean,Accuracy_std,Accuracy_95CI,F1_macro_mean,F1_macro_std,F1_macro_95CI,ROC_AUC_mean,ROC_AUC_std,ROC_AUC_95CI,PR_AUC_mean,PR_AUC_std,PR_AUC_95CI
0,Original,0.975667,0.002357,0.002927,0.975665,0.002358,0.002927,0.994992,0.000596,0.000740,0.993855,0.002207,0.002740
1,NoiseOnly,0.966000,0.005706,0.007085,0.965992,0.005716,0.007097,0.992419,0.002767,0.003436,0.991274,0.002932,0.003641
2,BinningOnly,0.987000,0.001667,0.002069,0.986999,0.001667,0.002070,0.998809,0.000184,0.000229,0.998857,0.000228,0.000283
3,MaskingOnly,0.958933,0.007021,0.008718,0.958909,0.007051,0.008755,0.987371,0.004894,0.006077,0.978945,0.011311,0.014044
4,PermutationOnly,0.491933,0.011663,0.014481,0.468336,0.016273,0.020206,0.491414,0.011867,0.014735,0.497042,0.009532,0.011835


In [ ]:
base = agg[agg["Dataset"] == "Original"].iloc[0]

agg["ΔAccuracy"] = agg["Accuracy_mean"] - base["Accuracy_mean"]
agg["ΔF1_macro"] = agg["F1_macro_mean"] - base["F1_macro_mean"]
agg["ΔROC_AUC"]  = agg["ROC_AUC_mean"]  - base["ROC_AUC_mean"]
agg["ΔPR_AUC"]   = agg["PR_AUC_mean"]   - base["PR_AUC_mean"]

agg

,Dataset,Accuracy_mean,Accuracy_std,Accuracy_95CI,F1_macro_mean,F1_macro_std,F1_macro_95CI,ROC_AUC_mean,ROC_AUC_std,ROC_AUC_95CI,PR_AUC_mean,PR_AUC_std,PR_AUC_95CI,ΔAccuracy,ΔF1_macro,ΔROC_AUC,ΔPR_AUC
0,Original,0.975667,0.002357,0.002927,0.975665,0.002358,0.002927,0.994992,0.000596,0.000740,0.993855,0.002207,0.002740,0.000000,0.000000,0.000000,0.000000
1,NoiseOnly,0.966000,0.005706,0.007085,0.965992,0.005716,0.007097,0.992419,0.002767,0.003436,0.991274,0.002932,0.003641,-0.009667,-0.009673,-0.002573,-0.002581
2,BinningOnly,0.987000,0.001667,0.002069,0.986999,0.001667,0.002070,0.998809,0.000184,0.000229,0.998857,0.000228,0.000283,0.011333,0.011334,0.003817,0.005002
3,MaskingOnly,0.958933,0.007021,0.008718,0.958909,0.007051,0.008755,0.987371,0.004894,0.006077,0.978945,0.011311,0.014044,-0.016733,-0.016756,-0.007621,-0.014910
4,PermutationOnly,0.491933,0.011663,0.014481,0.468336,0.016273,0.020206,0.491414,0.011867,0.014735,0.497042,0.009532,0.011835,-0.483733,-0.507329,-0.503578,-0.496813
